In [22]:
import requests
import pandas as pd
from io import BytesIO
import zipfile
import io
import os
import datetime
from pathlib import Path
import numpy as np
import sqlite3
from sqlalchemy import create_engine
from datetime import datetime


In [23]:
def operate(path,multiplier) :
    stockPrice = pd.read_csv(path)
    stockPrice  = stockPrice[((stockPrice['INSTRUMENT']=="OPTIDX") | (stockPrice['INSTRUMENT']=="OPTSTK"))]
    indicator = stockPrice[stockPrice['CONTRACTS'] > stockPrice['OPEN_INT']*multiplier]
    return indicator
    

In [24]:
multiplier=10
limit=10
engine = create_engine('sqlite:///C:\\Users\\Tathagat\\Desktop\\BhavCopy\\SqLite\\foo.db' , echo=False)
query =  "SELECT * FROM users where ((INSTRUMENT='OPTIDX' OR INSTRUMENT='OPTSTK') AND (CONTRACTS > OPEN_INT * " + str(multiplier) + ") AND (OPEN_INT> "+ str(limit) +" )) "
#temp = engine.execute("SELECT * FROM users where ((INSTRUMENT='OPTIDX' OR INSTRUMENT='OPTSTK') AND (CONTRACTS > OPEN_INT * multiplier) AND (OPEN_INT>10) ) ").fetchall()
queryResult = engine.execute(query).fetchall()
columnNames = engine.execute(" PRAGMA table_info(users) ")
columnNames = pd.DataFrame(columnNames)
columnNames = columnNames[1]
finalResult = pd.DataFrame(queryResult)
finalResult = finalResult.rename(columns=columnNames)
finalResult = finalResult.drop(finalResult.columns[[0,-1]] , axis=1)

In [25]:
finalResult

,INSTRUMENT,SYMBOL,EXPIRY_DT,STRIKE_PR,OPTION_TYP,OPEN,HIGH,LOW,CLOSE,SETTLE_PR,CONTRACTS,VAL_INLAKH,OPEN_INT,CHG_IN_OI,TIMESTAMP
0,OPTIDX,BANKNIFTY,10-Sep-2020,22400.0,CE,159.90,280.75,6.10,40.80,22466.20,3510915,19705288.96,283950,14300,10-SEP-2020
1,OPTIDX,BANKNIFTY,10-Sep-2020,22600.0,PE,319.95,385.95,39.85,149.05,22466.20,1152894,6546651.53,73625,27050,10-SEP-2020
2,OPTIDX,NIFTY,24-Sep-2020,6900.0,PE,0.25,1.00,0.20,0.20,0.20,1070,5537.45,75,75,21-SEP-2020
3,OPTIDX,BANKNIFTY,24-Sep-2020,20600.0,PE,20.00,156.40,7.10,84.65,20456.85,3071901,15843397.48,230300,425,24-SEP-2020
4,OPTIDX,BANKNIFTY,24-Sep-2020,20700.0,PE,34.00,256.35,20.00,190.35,20456.85,2782705,14441303.52,119900,-217950,24-SEP-2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420,OPTIDX,NIFTY,24-Sep-2020,6900.0,PE,0.25,1.00,0.20,0.20,0.20,1070,5537.45,75,75,21-SEP-2020
421,OPTIDX,BANKNIFTY,24-Sep-2020,20600.0,PE,20.00,156.40,7.10,84.65,20456.85,3071901,15843397.48,230300,425,24-SEP-2020
422,OPTIDX,BANKNIFTY,24-Sep-2020,20700.0,PE,34.00,256.35,20.00,190.35,20456.85,2782705,14441303.52,119900,-217950,24-SEP-2020
423,OPTIDX,BANKNIFTY,24-Sep-2020,20800.0,PE,70.00,356.30,43.70,292.15,20456.85,2174636,11363222.78,93025,-310200,24-SEP-2020


In [39]:
import timeit



In [40]:
start = timeit.default_timer()

temp['TIMESTAMP'] = temp['TIMESTAMP'].apply(lambda x: datetime.strptime(x, "%d-%b-%Y"))

stop = timeit.default_timer()

print('Time: ', stop - start) 

Time:  11.325016899999923


In [41]:
temp.sort_values('TIMESTAMP')

,INSTRUMENT,SYMBOL,EXPIRY_DT,STRIKE_PR,OPTION_TYP,OPEN,HIGH,LOW,CLOSE,SETTLE_PR,CONTRACTS,VAL_INLAKH,OPEN_INT,CHG_IN_OI,TIMESTAMP
24898,OPTIDX,NIFTY,29-Dec-2022,8300.0,CE,0.0,0.0,0.0,4484.95,5196.65,0,0.00,0,0,2020-01-01
24163,OPTIDX,NIFTY,24-Sep-2020,14600.0,PE,0.0,0.0,0.0,1889.95,1859.80,0,0.00,0,0,2020-01-01
24164,OPTIDX,NIFTY,24-Sep-2020,14700.0,PE,0.0,0.0,0.0,1876.60,1949.45,0,0.00,0,0,2020-01-01
24165,OPTIDX,NIFTY,31-Dec-2020,4600.0,CE,0.0,0.0,0.0,3955.65,7839.55,0,0.00,0,0,2020-01-01
24166,OPTIDX,NIFTY,31-Dec-2020,4700.0,CE,0.0,0.0,0.0,3896.80,7745.15,0,0.00,0,0,2020-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21029,OPTIDX,NIFTY,15-Oct-2020,11000.0,PE,195.6,314.3,195.6,305.40,305.40,47,397.06,1125,-1200,2020-09-24
21028,OPTIDX,NIFTY,15-Oct-2020,10950.0,PE,0.0,0.0,0.0,432.05,347.90,0,0.00,0,0,2020-09-24
21027,OPTIDX,NIFTY,15-Oct-2020,10900.0,PE,0.0,0.0,0.0,411.60,319.75,0,0.00,0,0,2020-09-24
21121,OPTIDX,NIFTY,22-Oct-2020,12050.0,CE,0.0,0.0,0.0,377.60,32.35,0,0.00,0,0,2020-09-24
